In [10]:
!pip install prettytable


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import torch as th
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from envs.bidding import BiddingEnv  # Assuming this is your custom environment
from tabulate import tabulate
import numpy as np
# from prettytable import PrettyTable

In [33]:
log_dir = "./runs/baselines"


env = BiddingEnv(render_mode='human')

# Wrap with VecNormalize for observation and reward normalization
vec_env = DummyVecEnv([lambda: env])  # Single process
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)


# Load the pretrained model
model = SAC.load("./runs/baselines/SAC/best_model", env=vec_env)

In [40]:
# Reset the environment
obs = vec_env.reset()

print(obs['robot_0_self_state'])
# print(obs.robot_1_self_state)

# for item in obs:
#     if 'self_state' in item: 
#         print('self_state = ', item)

# Run for a specified number of episodes or steps
for episode in range(1):
    done = False
    total_reward = 0

    counter = 0
    
    while not done:

        counter += 1

        print(obs['robot_0_self_state'])

        # Get action from the model
        action, _ = model.predict(obs, deterministic=True)

        action_array = np.array(action).reshape(9, 6)

        # print(f"Step {counter} has Action:")
        # table = tabulate(action_array, headers=[f"Robot {i+1}" for i in range(6)], tablefmt="fancy_grid", showindex="always")
        # print(table)


        # Step the environment
        obs, reward, done, info = vec_env.step(action)
        
        # Render the environment (if your environment supports rendering)
        vec_env.render(mode='human')
        
        total_reward += reward
    
    print(f"Episode {episode + 1} Total Reward: {total_reward}")

# Close the environment when done
vec_env.close()


[[ 1.0730231 -1.1366774  1.185443 ]]
[[ 1.0730231 -1.1366774  1.185443 ]]
[[ 1.0561998 -1.1177198  1.1647209]]
[[ 1.0401437 -1.0996803  1.1450487]]
[[ 1.0247983 -1.0824869  1.1263409]]
[[ 1.0101126 -1.0660754  1.108521 ]]
[[ 0.9960407 -1.0503886  1.091521 ]]
[[ 0.98254097 -1.0353744   1.0752798 ]]
[[ 0.9695757 -1.0209862  1.0597428]]
[[ 0.95711046 -1.0071816   1.0448604 ]]
[[ 0.94511396 -0.9939223   1.0305878 ]]
Episode 1 Total Reward: [2.6313264]


In [1]:
# custom environment
from envs.bidding import BiddingEnv

env = BiddingEnv()

env.render() # choose from verbose, bids, plot

obs = env.reset()

# print('Observation space: \n', env.observation_space)
# print('Action space: \n', env.action_space)
# print('Random Action (Bid Matrix): \n', env.action_space.sample())
# print('Random Observation: \n', env.observation_space.sample())

env.render(mode='human')

for step in range(10):
    
    obs, reward, done, truncated, info = env.step(env.action_space.sample())
    env.render(mode='human')
    print(f'\n Step {step + 1} with reward = {reward}')

    if done:
        env.render(mode='human')
        print("Completed, final reward =", reward)
        break

print('optimal reward = ', env.optimal_reward())

Step: 0

Robots:
  Robot 1: Robot at (3, 1) with type A-humanbot
  Robot 2: Robot at (0, 2) with type B-navbot
  Robot 3: Robot at (5, 0) with type A-humanbot
  Robot 4: Robot at (4, 8) with type A-humanbot
  Robot 5: Robot at (2, 0) with type A-humanbot
  Robot 6: Robot at (9, 4) with type A-humanbot
  Robot 7: Robot at (5, 3) with type A-humanbot
  Robot 8: Robot at (6, 9) with type B-navbot
  Robot 9: Robot at (8, 1) with type A-humanbot

Tasks:
  Task 1: Task at (6, 9) with prize 97 and type C-specialty
  Task 2: Task at (2, 7) with prize 5 and type A-manipulation
  Task 3: Task at (2, 6) with prize 36 and type A-manipulation
  Task 4: Task at (0, 7) with prize 2 and type C-specialty
  Task 5: Task at (6, 7) with prize 67 and type B-transport
  Task 6: Task at (4, 4) with prize 57 and type A-manipulation

Bidding Matrix:
╒══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│   Task 1 │   Task 2 │   Task 3 │   Task 4 │   Task 5 │   Task 6 │
╞══════════╪══════════╪════

In [2]:
from stable_baselines3.common.env_checker import check_env
from envs.bidding import BiddingEnv

env = BiddingEnv()

obs = env.reset()

print("Observation Output:", obs)
print("Observation Space:", env.observation_space)

env = BiddingEnv()

# check stable baselines compatibility
check_env(env, warn=True)


# print('optimal reward = ', env.optimal_reward())

Observation Output: ({'robot_positions': array([[5, 7],
       [7, 9],
       [8, 4],
       [3, 3],
       [7, 8],
       [6, 3],
       [6, 2],
       [3, 2],
       [1, 5]], dtype=int32), 'robot_types': array([2, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32), 'task_positions': array([[8, 5],
       [1, 6],
       [4, 6],
       [6, 5],
       [1, 9],
       [9, 0]], dtype=int32), 'task_prizes': array([30, 13, 35, 17,  1, 87], dtype=int32), 'task_types': array([1, 1, 2, 0, 2, 2], dtype=int32)}, {})
Observation Space: Dict('robot_positions': Box(0, 9, (9, 2), int32), 'robot_types': Box(0, 2, (9,), int32), 'task_positions': Box(0, 9, (6, 2), int32), 'task_prizes': Box(1, 100, (6,), int32), 'task_types': Box(0, 2, (6,), int32))


/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation robot_positions has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation task_positions has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/Users/devg/robo

Error while checking key=robots: The observation returned by the `reset()` method does not match the given observation space Dict('positions': Box(0, 9, (9, 2), int32), 'types': Box(0, 2, (9,), int32))